In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from shapely.geometry import Polygon, mapping
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import earthpy as et
import earthpy.plot as ep

__Idea:__ Train a classifier on the pixels in the mask of the training data (if it exists?), then classify each pixel in the eval data as tree or not.

In [40]:
dtm_train_path = "../../data/data/dtm_train.tif"
dsm_train_path = "../../data/data/dsm_train.tif"
ortho_train_path = "../../data/data/ortho_train.tif"
ortho_eval_path = "../../data/eval_data/ortho_eval.tif"

In [20]:
with rio.open(dtm_train_path) as dtm:
    print(dtm.crs)

EPSG:32617


In [21]:
dtm.bounds

BoundingBox(left=274804.78000006, bottom=4603079.993190537, right=275062.7799999895, top=4603433.35924399)

In [22]:
dtm.res

(0.193403298298, 0.193413275015)

In [ ]:
with rio.open(lidar_dem_path) as src:
    # Convert / read the data into a numpy array
    # masked = True turns `nodata` values to nan
    lidar_dem_im = src.read(1, masked=True)
    
    # Create a spatial extent object using rio.plot.plotting
    spatial_extent = rio.plot.plotting_extent(src)

In [26]:
with rio.open(ortho_train_path) as ortho:
    print(ortho.meta)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 5160, 'height': 7068, 'count': 4, 'crs': CRS.from_epsg(32617), 'transform': Affine(0.0499999999864, 0.0, 274804.78000006,
       0.0, -0.0499951971494, 4603433.35924399)}


In [33]:
with rio.open(ortho_train_path) as ortho:
    ortho_data = ortho.read(masked = True)

In [31]:
with rio.open(ortho_train_path) as ortho:
    ortho_masks = ortho.read_masks()

In [41]:
with rio.open(ortho_eval_path) as ortho_eval:
    ortho_eval_masks = ortho_eval.read_masks()

Training data does not have a mask for trees already. Need to create one first.

List of things to do:
1. Check if all rasters are in same projection and have same dimensions
2. Make new mask for canopy height (CHM = DEM - DTM)
3. Check if CHM for trees and buildings can be distinguished (threshold for height?)
4. Calculate green chromatic coordinate (GCC = green/(green + red + blue)), maybe 0.33-0.4 will be trees
5. Make tree mask 
6. Train ML model (CNN? Decision trees?)

In [45]:
paths = [dsm_train_path, dtm_train_path, ortho_train_path, ortho_eval_path]

for each in paths:
    with rio.open(each) as src:
        print(src.crs, src.bounds, src.width, src.height, src.res)

EPSG:32617 BoundingBox(left=274804.78000006, bottom=4603079.993190537, right=275062.7799999895, top=4603433.35924399) 1334 1827 (0.193403298298, 0.193413275015)
EPSG:32617 BoundingBox(left=274804.78000006, bottom=4603079.993190537, right=275062.7799999895, top=4603433.35924399) 1334 1827 (0.193403298298, 0.193413275015)
EPSG:32617 BoundingBox(left=274804.78000006, bottom=4603079.993190538, right=275062.7799999898, top=4603433.35924399) 5160 7068 (0.0499999999864, 0.0499951971494)
EPSG:32617 BoundingBox(left=274804.78000006, bottom=4603079.993190538, right=275062.7799999898, top=4603433.35924399) 5160 7068 (0.0499999999864, 0.0499951971494)
